In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn import svm
import glob

from datetime import datetime, date

In [2]:
def computeHoG(image, winSize, blockSize, blockStride, cellSize,
                               nbins, RESIZE_DIM, cell_size, block_size):
    
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

    n_cells = (RESIZE_DIM // cell_size[0], RESIZE_DIM // cell_size[1])

    # Compute HoG features
    hog_feats = hog.compute(image)
    hog_feats = hog_feats.reshape(n_cells[1]-block_size[1]+1,
                                  n_cells[0]-block_size[0]+1,
                                  block_size[0], block_size[1], nbins)
    hog_feats = hog_feats.transpose((1, 0, 2, 3, 4))  # index blocks by rows first

    # hog_feats now contains the gradient amplitudes for each direction,
    #for each cell of its group for each group.
    # Indexing is by rows then columns.

    # computation for BlockNorm
    gradients = np.full((n_cells[0], n_cells[1], 8), 0, dtype=float)
    cell_count = np.full((n_cells[0], n_cells[1], 1), 0, dtype=int)

    for off_y in range(block_size[0]):
        for off_x in range(block_size[1]):
            gradients[off_y:n_cells[0] - block_size[0] + off_y + 1,
                      off_x:n_cells[1] - block_size[1] + off_x + 1] += \
                hog_feats[:, :, off_y, off_x, :]
            cell_count[off_y:n_cells[0] - block_size[0] + off_y + 1,
                       off_x:n_cells[1] - block_size[1] + off_x + 1] += 1

    # Average gradients
    gradients /= cell_count

    gradients = gradients.reshape((1, -1))
    return gradients

def resizeImage(image, dimension):
    imy, imx = image.shape[0:2]
    M = np.float32([[dimension/imx, 0, 0], [0, dimension/imy, 0]])
    image = cv2.warpAffine(image, M, (RESIZE_DIM, RESIZE_DIM))
    return image

In [3]:
#displaay
startGlobal = datetime.time(datetime.now())

# #testing
categoriesTest  = [
#      "car",
#      "background",
#      "pickup_truck",
#     "articulated_truck",
#     "bus",
     "work_van",
     "pedestrian",
    "single_unit_truck",
    "bicycle",
    "motorcycle",
    "non-motorized_vehicle"]
#endTesting
#^We notice that runtime is minimized of operations occur on the largest categories first
numCategories = len(categoriesTest)

#TEMP
sumIm = 0
for i in range(numCategories):
    path = ("MIO-TCD-Classification/train/%s/*.jpg" % categoriesTest[i])
    numImages = len(glob.glob(path))
    sumIm = sumIm + numImages
#End TEMP

RESIZE_DIM = 64
# totalNumIm = 519164 #MAY GIVE MEMORY ERROR
#testing
totalNumIm = sumIm
#End testing
globalImCtr = 0

#HoG variables
#Raw variables
cell_size = (4,4)
block_size = (4, 4)

#variables for computation
winSize = (RESIZE_DIM // cell_size[1] * cell_size[1], RESIZE_DIM // cell_size[0] * cell_size[0] )
blockSize = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
blockStride = (cell_size[1], cell_size[0])
cellSize = (cell_size[1], cell_size[0])
nbins = 8
#End HoG variables

totalNumFeatures = RESIZE_DIM // cell_size[1] * RESIZE_DIM // cell_size[0] * nbins

#array initialization
labelsArray = np.zeros(totalNumIm, dtype=np.int)
featuresArray = np.zeros((totalNumIm, totalNumFeatures))
indexArray = np.arange(totalNumIm, dtype=np.int)
np.random.shuffle(indexArray)         

# Read input images category by category
for i in range(numCategories):
    start = datetime.time(datetime.now())

    #specifying path to images
    path = ("MIO-TCD-Classification/train/%s/*.jpg" % categoriesTest[i])

    #Variable initializations
    categoryImgCnt = 0
    nextPctg = 5
    numImages = len(glob.glob(path))

    #display
    print("Loading Images for category:    >>>" + categoriesTest[i] + "<<<    " + str(numImages) + " images")
    print("    Percentage Completed: 0%-", end='')

    # for each image in the category folder
    for file in glob.glob(path):
        #read, convert color, get features, stor info
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #resizing IF NECESSARY
        #TRY to do HoG without resizing image. to be discussed
        image = resizeImage(image, RESIZE_DIM)
        #end resize
        
        #Computing features
        #HoG features
        gradients = computeHoG(image, winSize, blockSize, blockStride, cellSize,
                               nbins, RESIZE_DIM, cell_size, block_size)

        randomIndex = indexArray[globalImCtr]
        featuresArray[randomIndex] = gradients
        labelsArray[randomIndex] = i
        #End Feature computation
        
        #variable update
        globalImCtr = globalImCtr + 1
        categoryImgCnt = categoryImgCnt + 1
        
        #display
        if (int(categoryImgCnt*100/numImages)) == nextPctg:
            print(str(nextPctg) + "%", end='')
            if nextPctg != 100 : print("-", end='')
            nextPctg = nextPctg + 5
    #display        
    print("")
    end = datetime.time(datetime.now())
    print("    Duration: " + str(datetime.combine(date.today(), end) - datetime.combine(date.today(), start)))

#display
endGlobal = datetime.time(datetime.now())
duration = datetime.combine(date.today(), endGlobal) - datetime.combine(date.today(), startGlobal)
print(">Total duration: " + str(duration))
print(labelsArray[:100])

Loading Images for category:    >>>single_unit_truck<<<    5120 images
    Percentage Completed: 0%-5%-10%-15%-20%-25%-30%-35%-40%-45%-50%-55%-60%-65%-70%-75%-80%-85%-90%-95%-100%
    Duration: 0:00:07.185048
Loading Images for category:    >>>bicycle<<<    2284 images
    Percentage Completed: 0%-5%-10%-15%-20%-25%-30%-35%-40%-45%-50%-55%-60%-65%-70%-75%-80%-85%-90%-95%-100%
    Duration: 0:00:02.135318
Loading Images for category:    >>>motorcycle<<<    1982 images
    Percentage Completed: 0%-5%-10%-15%-20%-25%-30%-35%-40%-45%-50%-55%-60%-65%-70%-75%-80%-85%-90%-95%-100%
    Duration: 0:00:01.821099
Loading Images for category:    >>>non-motorized_vehicle<<<    1751 images
    Percentage Completed: 0%-5%-10%-15%-20%-25%-30%-35%-40%-45%-50%-55%-60%-65%-70%-75%-80%-85%-90%-95%-100%
    Duration: 0:00:01.983694
>Total duration: 0:00:13.172000
[0 0 1 0 0 0 2 3 0 0 0 0 2 0 1 2 0 0 0 0 2 2 1 0 0 0 1 3 1 1 3 0 1 1 2 2 0
 3 0 3 1 0 0 1 2 0 2 3 0 1 3 2 3 0 2 3 1 0 0 3 3 0 3 1 0 0 0 3 2 0 0 0

In [6]:
numSubset = 10
subsetSize = featuresArray.shape[0]//numSubset

#TESTING
print("classification begins")
for subsetIndex in range(numSubset):
    classifier = svm.SVC(C = 100.) #By default: kernel  = 'rfb'
    classifier.fit(np.concatenate((featuresArray[:subsetIndex*subsetSize], 
                                   featuresArray[(subsetIndex+1)*subsetSize:]), axis = 0)
                   , np.concatenate((labelsArray[:subsetIndex*subsetSize], 
                                   labelsArray[(subsetIndex+1)*subsetSize:]), axis = 0))
    #Takes long because too many features per image (cuz image large)
    #PREDICTION
    Accuracy = 0
    prediction = classifier.predict(featuresArray[subsetIndex*subsetSize:(subsetIndex+1)*subsetSize])[0]
    Accuracy = np.sum(prediction == labelsArray[subsetIndex*subsetSize:(subsetIndex+1)*subsetSize])
    Accuracy /= (subsetSize/100)
    print("Accuracy for subset %i is %i percent" % (subsetIndex, Accuracy))
print("classification finished")
#END TESTING

classification begins
Accuracy for subset 0 is 46 percent
classification finished
